# Reccomendation Engine for Location 

In [42]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries are imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries are imported.


## load Toronto postal codes 

In [43]:
# import the data created in week 3
# Loading the dataset which is about postal codes in Toronto
# This dataset was created in week 3. 
df_toronto = pd.read_csv(r'C:\Users\w5080797\Documents\Personal\IBM_qualification\df_neighborhoods_coordinates.csv')
df_toronto.head()


,Unnamed: 0,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,0,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
1,1,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que...",M8Y,43.636258,-79.498509
2,2,East York,Leaside,M4G,43.709060,-79.363452
3,3,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K,43.647177,-79.381576
4,4,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476


In [5]:
# Save the data for later
df_toronto.to_csv('df_toronto.csv')

## create a map of Toronto using postal codes

In [44]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

## concentrate on the "Scarorough" Borough in Toronto 

In [7]:
# df_toronto['Borough'] == 'Scarborough'

# selecting only neighborhoods regarding to "Scarborough" borough.
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
scarborough_data.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
1,Scarborough,"Cliffcrest, Cliffside",M1M,43.716316,-79.239476
2,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
3,Scarborough,"Ionview, Kennedy Park",M1K,43.727929,-79.262029
4,Scarborough,"Maryvale, Wexford",M1R,43.750071,-79.295849


## create a map of Create a Map of Scarborough and Its Neighbourhoods

In [45]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of "Scarborough" are: 43.773077, -79.257774.


In [9]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [10]:
# @hiddel_cell
CLIENT_ID = 'OVNJKI2111SKDXHSVBMPEJ1YKQYJ2KHJ4QUXKYGIEKKUFXDA' # your Foursquare ID
CLIENT_SECRET = 'LPMXE1UZXFZ5GBEMWI2BXDZYHJFCKZOHSM5TFDKQCZJAC1QX' # your Foursquare Secret
VERSION = '20170416' # Foursquare API version

## search for venues in the neighborhoods inside "Scarborough"

In [11]:
print('Search different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Postal Code']),
                                                   list(scarborough_data['Neighbourhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Search different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1P (and Neighborhoods Dorset Park, Scarborough Town Centre, Wexford Heights) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffcrest, Cliffside) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Rouge, Malvern) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1K (and Neighborhoods Ionview, Kennedy Park) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1R (and Neighborhoods Maryvale, Wexford) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1N (and Neighborhoods Birch Cliff) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1E (and Neighborhoods Morningside, West Hill) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1X (and Neighborhoods Upper Rouge) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
10.
Data is Obtained, for the Postal Co


## store the results to avoid reconnecting to foursquare everytime them

In [12]:
import pickle
with open("Scarborough_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Scarborough_foursquare_dataset, fp)
print('Received Data from Internet is Saved .')

Received Data from Internet is Saved .


In [13]:
with open("Scarborough_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Scarborough_foursquare_dataset = pickle.load(fp)
# print(type(Scarborough_foursquare_dataset))
# Scarborough_foursquare_dataset

## Data cleaning


In [14]:
# This function is created to connect to the saved list which is the received database. It will extract each venue 
# for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venues in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [15]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venues in Coordination "M1P" Posal Code and "Dorset Park, Scarborough Town Centre, Wexford Heights" Negihborhood(s) is:
49
Number of Venues in Coordination "M1M" Posal Code and "Cliffcrest, Cliffside" Negihborhood(s) is:
12
Number of Venues in Coordination "M1B" Posal Code and "Rouge, Malvern" Negihborhood(s) is:
15
Number of Venues in Coordination "M1K" Posal Code and "Ionview, Kennedy Park" Negihborhood(s) is:
24
Number of Venues in Coordination "M1R" Posal Code and "Maryvale, Wexford" Negihborhood(s) is:
30
Number of Venues in Coordination "M1N" Posal Code and "Birch Cliff" Negihborhood(s) is:
13
Number of Venues in Coordination "M1E" Posal Code and "Morningside, West Hill" Negihborhood(s) is:
23
Number of Venues in Coordination "M1X" Posal Code and "Upper Rouge" Negihborhood(s) is:
0
Number of Venues in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is:
11
Number of Venues in Coordination "M1L" Posal Code and "Clairlea, Golden Mile, Oakridge" Negi

## display Venues for Each Neighborhood in Scarborough


In [16]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kairali,This spot is popular,Indian Restaurant,429
1,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kim Kim restaurant,This spot is popular,Chinese Restaurant,478
2,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Starbucks,This spot is popular,Coffee Shop,600
3,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Harry's Drive-In,This spot is popular,Burger Joint,965
4,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Spicy Dragon,This spot is popular,Asian Restaurant,922


In [17]:
scarborough_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
324,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Alexmuir Park,This spot is popular,Park,760
325,M1V,"Agincourt North, Milliken",43.815252,-79.284577,DaanGo Cake Lab,This spot is popular,Bakery,809
326,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Tang Home Style Lamb & Beef 老汤家,This spot is popular,Chinese Restaurant,825
327,M1V,"Agincourt North, Milliken",43.815252,-79.284577,South China Noodles 桂林粉麵食店,This spot is popular,Chinese Restaurant,844
328,M1V,"Agincourt North, Milliken",43.815252,-79.284577,Silverland Centre 銀星中心,This spot is popular,Shopping Mall,928


## Saving a Cleaned Version of DataFrame as the Results from Foursquare

In [18]:
scarborough_venues.to_csv('scarborough_venues.csv')

In [19]:
scarborough_venues = pd.read_csv('scarborough_venues.csv')

## summary of the neighborhoods data


In [20]:
neigh_list = list(scarborough_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Scarborough:')
print(len(neigh_list))
print('List of Neighborhoods inside Scarborough:')
neigh_list

Number of Neighborhoods inside Scarborough:
14
List of Neighborhoods inside Scarborough:


['Dorset Park, Scarborough Town Centre, Wexford Heights',
 'Cliffcrest, Cliffside',
 'Rouge, Malvern',
 'Ionview, Kennedy Park',
 'Maryvale, Wexford',
 'Birch Cliff',
 'Morningside, West Hill',
 'Scarborough Village',
 'Clairlea, Golden Mile, Oakridge',
 'Woburn',
 'Agincourt',
 "Tam O'Shanter",
 'Highland Creek, Rouge Hill, Port Union',
 'Agincourt North, Milliken']

In [21]:
# more summary information
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
# neigh_venue_summary.drop(columns = ['Unnamed: 0'], inplace = True)

In [22]:
# scarborough_venues.drop(columns = ['Unnamed: 0'], inplace = True)

In [23]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())


There are 105 uniques categories.
Here is the list of different categories:


['Indian Restaurant',
 'Chinese Restaurant',
 'Coffee Shop',
 'Burger Joint',
 'Asian Restaurant',
 'Electronics Store',
 'Bakery',
 'Latin American Restaurant',
 'Gym / Fitness Center',
 'Pharmacy',
 'Pet Store',
 'Grocery Store',
 'Italian Restaurant',
 'Vietnamese Restaurant',
 'Bank',
 'Beer Store',
 'Sandwich Place',
 'Fast Food Restaurant',
 'Fried Chicken Joint',
 'Thrift / Vintage Store',
 'Pizza Place',
 'Furniture / Home Store',
 'Wings Joint',
 'Bowling Alley',
 'American Restaurant',
 'Bus Station',
 'Automotive Shop',
 'Discount Store',
 'Light Rail Station',
 'Plaza',
 'Park',
 'Intersection',
 'Restaurant',
 'Dessert Shop',
 'Beach',
 'Sports Bar',
 'Cajun / Creole Restaurant',
 'Spa',
 'Caribbean Restaurant',
 'Paper / Office Supplies Store',
 'Hobby Shop',
 'Martial Arts Dojo',
 'Fruit & Vegetable Store',
 'Gym',
 'Department Store',
 'Convenience Store',
 'Hockey Arena',
 'Metro Station',
 'Bus Line',
 'Rental Car Location',
 'Korean Restaurant',
 'Fish Market',
 'Bre

## Encoding the "categroies" Column into Every Unique Categorical Feature.

In [24]:

scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()


,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,American Restaurant,Asian Restaurant,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Hockey Arena,Indian Chinese Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Noodle House,Optical Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint
0,0,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kairali,This spot is popular,429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kim Kim restaurant,This spot is popular,478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Starbucks,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Harry's Drive-In,This spot is popular,965,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Spicy Dragon,This spot is popular,922,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Selecting  Impotant Features for the Property Developer

In [27]:
important_features = [
 'Electronics Store',
 'Bakery',
 
 'Gym / Fitness Center',
 'Pharmacy',

 'Grocery Store',
 
 'Bank',
 'Beer Store',
 
 'Light Rail Station',
 
 'Furniture / Home Store',
 
 'Bowling Alley',
 'Bus Station',
 'Automotive Shop',
 'Discount Store',
 'Clothing Store',
 'Plaza',
 'Park',
 'Intersection',
 'Beach',
 'Sports Bar',
 'Supermarket',
 'Pool',
 'Pool Hall',
 'Shopping Mall',
 'Skating Rink',
 'Badminton Court',
 'Motorcycle Shop',
 'Optical Shop',
 'General Entertainment',
 'Gym',
 'Diner',
 'College Stadium',
 'Convenience Store',
 'Gym Pool',
 'Playground',
 'Spa',
 'Paper / Office Supplies Store',
  'Martial Arts Dojo',
 'Fruit & Vegetable Store',
 'Department Store',
 'Hockey Arena',
 'Metro Station',
 'Rental Car Location', 
 'Shop & Service',
 'Bus Line',
 'Soccer Field',
 'Pub',
 'Train Station',
 'Golf Course',
  'Video Game Store',
  'Bar',
  'Liquor Store',
  'Neighborhood']
    


## Updating DataFrame and Grouping the Data by Neighborhoods

In [28]:
scarborough_onehot = scarborough_onehot[important_features].groupby(
    'Neighborhood').sum()


scarborough_onehot.head()

,Electronics Store,Bakery,Gym / Fitness Center,Pharmacy,Grocery Store,Bank,Beer Store,Light Rail Station,Furniture / Home Store,Bowling Alley,Bus Station,Automotive Shop,Discount Store,Clothing Store,Plaza,Park,Intersection,Beach,Sports Bar,Supermarket,Pool,Pool Hall,Shopping Mall,Skating Rink,Badminton Court,Motorcycle Shop,Optical Shop,General Entertainment,Gym,Diner,College Stadium,Convenience Store,Gym Pool,Playground,Spa,Paper / Office Supplies Store,Martial Arts Dojo,Fruit & Vegetable Store,Department Store,Hockey Arena,Metro Station,Rental Car Location,Shop & Service,Bus Line,Soccer Field,Pub,Train Station,Golf Course,Video Game Store,Bar,Liquor Store
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,2,0,1,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,2,1,1,3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Agincourt North, Milliken",0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Birch Cliff,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Clairlea, Golden Mile, Oakridge",0,2,0,0,1,1,1,0,0,0,1,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,1,1,2,0,2,0,0,0,0,0,0,0,0,1,0,0,2,1,1,0,0,0,0,0
"Cliffcrest, Cliffside",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
scarborough_onehot.to_csv('scarborough_onehot.csv')

In [30]:
scarborough_onehot = pd.read_csv('scarborough_onehot.csv',index_col=0)

## group similar categories together

**we will group intuitively assuming categories which are similar have equal importance to the buyer**

In [31]:
feat_name_list = list(scarborough_onehot.columns)
bar_Pub_list = []

for counter, value in enumerate (feat_name_list):
    if value in ('Beer Store', 'Pub', 'Bar','Liquor Store' ) != (-1):
        bar_Pub_list.append(value)
        
scarborough_onehot['Total bar_Pub'] = scarborough_onehot[bar_Pub_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = bar_Pub_list)        
        
        
feat_name_list = list(scarborough_onehot.columns)
stores_list = []

for counter, value in enumerate (feat_name_list):
    if value in ( 'Store', 'shop', 'Bar','Shopping Mall') != (-1):
        stores_list.append(value)
        
scarborough_onehot['Total stores_list'] = scarborough_onehot[stores_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = stores_list)



feat_name_list = list(scarborough_onehot.columns)
transportation_list = []

for counter, value in enumerate (feat_name_list):
    if value in ('Light Rail Station','Bus Station','Rental Car Location','Bus Line','Train Station','Intersection') != (-1):
        transportation_list.append(value)
        
scarborough_onehot['Total transportation_list'] = scarborough_onehot[transportation_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = transportation_list)


        
feat_name_list = list(scarborough_onehot.columns)
sports_facilities = []

for counter, value in enumerate (feat_name_list):
    if value in ('Gym / Fitness Center','Bowling Alley','Skating Rink','Gym','Gym Pool','Playground','Hockey Arena','Soccer Field','Golf Course', 'Martial Arts Dojo' ) != (-1):
        sports_facilities.append(value)
        
scarborough_onehot['Total sports_facilities'] = scarborough_onehot[sports_facilities].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = sports_facilities)



feat_name_list = list(scarborough_onehot.columns)
special_services = []

for counter, value in enumerate (feat_name_list):
    if value in ('Pharmacy','Bank','Spa','Laundromat',) != (-1):
        special_services.append(value)
        
scarborough_onehot['Total special_services'] = scarborough_onehot[special_services].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = special_services)


## The cleaned dataset for neighborhoods in Scarborrough

In [32]:
scarborough_onehot

,Electronics Store,Bakery,Grocery Store,Furniture / Home Store,Automotive Shop,Discount Store,Clothing Store,Plaza,Park,Beach,Sports Bar,Supermarket,Pool,Pool Hall,Badminton Court,Motorcycle Shop,Optical Shop,General Entertainment,Diner,College Stadium,Convenience Store,Paper / Office Supplies Store,Fruit & Vegetable Store,Department Store,Metro Station,Shop & Service,Video Game Store,Total bar_Pub,Total stores_list,Total transportation_list,Total sports_facilities,Total special_services
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,2,1,0,0,1,1,0,1,0,0,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2
"Agincourt North, Milliken",0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,2
Birch Cliff,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,3,1
"Clairlea, Golden Mile, Oakridge",0,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,2,0,2,0,0,0,1,0,0,2,0,6,2,1
"Cliffcrest, Cliffside",0,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Dorset Park, Scarborough Town Centre, Wexford Heights",4,2,1,2,2,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,2,2
"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
"Ionview, Kennedy Park",0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,3,1,1
"Maryvale, Wexford",0,1,2,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,2,1,0


## Group the neighbourhoods into 5 clusters using k-means

In [33]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

## Show  centres of the clusters

In [34]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['C1','C2','C3','C4','C5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Electronics Store,Bakery,Grocery Store,Furniture / Home Store,Automotive Shop,Discount Store,Clothing Store,Plaza,Park,Beach,Sports Bar,Supermarket,Pool,Pool Hall,Badminton Court,Motorcycle Shop,Optical Shop,General Entertainment,Diner,College Stadium,Convenience Store,Paper / Office Supplies Store,Fruit & Vegetable Store,Department Store,Metro Station,Shop & Service,Video Game Store,Total bar_Pub,Total stores_list,Total transportation_list,Total sports_facilities,Total special_services,Total Sum
C3,4.000000,2.000000,1.000000,2.000000,2.000000e+00,1.0,0.000000e+00,1.000000e+00,1.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000,3.000000,2.000000,2.000000,22.000000
C4,0.000000,2.000000,1.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.0,0.000000,6.000000,2.000000,1.000000,21.000000
C5,0.000000,1.666667,0.666667,0.000000,0.000000e+00,1.0,3.333333e-01,0.000000e+00,1.666667e+00,0.000000,0.000000,6.666667e-01,3.333333e-01,3.333333e-01,3.333333e-01,3.333333e-01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.333333e-01,3.333333e-01,0.0,1.666667,0.666667,0.666667,2.000000,13.000000
C1,0.000000,0.333333,1.666667,0.000000,0.000000e+00,1.0,0.000000e+00,0.000000e+00,1.110223e-16,0.000000,0.333333,6.666667e-01,0.000000e+00,0.000000e+00,3.333333e-01,0.000000e+00,3.333333e-01,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,3.333333e-01,3.333333e-01,0.000000e+00,0.000000e+00,1.0,0.000000,2.000000,0.666667,1.000000,10.666667
C2,0.166667,0.000000,0.000000,0.166667,2.775558e-17,0.0,1.387779e-17,1.387779e-17,1.000000e+00,0.333333,0.166667,5.551115e-17,1.387779e-17,1.387779e-17,2.775558e-17,1.387779e-17,1.387779e-17,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,1.387779e-17,2.775558e-17,1.387779e-17,1.387779e-17,0.0,0.000000,0.166667,1.166667,0.333333,4.500000


**Results:**<br>
**the best  group is Cluster  4** <br>
**the second best group is Cluster 1 because as much as total points are less than those of Cluster 3, there are no pubs/clubs in this area** <br>
**the third best is Cluster 3 which was lowered by the pub/clusters in the area** <br>

## Match the clusters to the Scarborough Neighbourhoods**  



In [35]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,5
1,"Agincourt North, Milliken",5
2,Birch Cliff,2
3,"Clairlea, Golden Mile, Oakridge",4
4,"Cliffcrest, Cliffside",2
5,"Dorset Park, Scarborough Town Centre, Wexford ...",3
6,"Highland Creek, Rouge Hill, Port Union",2
7,"Ionview, Kennedy Park",1
8,"Maryvale, Wexford",1
9,"Morningside, West Hill",1


## Interpreting results

In [37]:
print( " The best Neighborhoods are/is  ")
neigh_summary[neigh_summary['Group'] == 4]

 The best Neighborhoods are/is  


,Neighborhood,Group
3,"Clairlea, Golden Mile, Oakridge",4


In [38]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 4]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Neighborhood': 'Clairlea, Golden Mile, Oakridge',
 'Neighborhood Latitude': 43.711111700000004,
 'Neighborhood Longitude': -79.284577200000001,
 'Postal Code': 'M1L'}

In [39]:
print ("The second best neighborhood are/is")
neigh_summary[neigh_summary['Group'] == 1]

The second best neighborhood are/is


,Neighborhood,Group
7,"Ionview, Kennedy Park",1
8,"Maryvale, Wexford",1
9,"Morningside, West Hill",1


In [40]:
print ("The third best neighborhood are/is")
neigh_summary[neigh_summary['Group'] == 3]

The third best neighborhood are/is


,Neighborhood,Group
5,"Dorset Park, Scarborough Town Centre, Wexford ...",3


## conclusion:

**Although our data did not have information about schools and clinics. we used a selection of venues and amneties that most home buyers still consider when
purchasing houses, these features are common among family oriented buyers; they consider convenience and lifestyle**